# 나이브 베이즈

In [280]:
import pandas as pd

df = pd.read_csv('sms_spam.csv', encoding='ISO-8859-1' )

In [281]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5559 entries, 0 to 5558
Data columns (total 2 columns):
type    5559 non-null object
text    5559 non-null object
dtypes: object(2)
memory usage: 86.9+ KB


In [282]:
df.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or Â£10,000..."
4,spam,okmail: Dear Dave this is your final notice to...


In [283]:
df['text'][3]

'complimentary 4 STAR Ibiza Holiday or Â£10,000 cash needs your URGENT collection. 09066364349 NOW from Landline not to lose out! Box434SK38WP150PPM18+'

In [284]:
text = df['text'][3]
text = re.sub('\W', ' ', text)
text = re.sub('\d', '', text)
text = re.sub('[ \t]+' , ' ', text)
text

'complimentary STAR Ibiza Holiday or Â cash needs your URGENT collection NOW from Landline not to lose out BoxSKWPPPM '

In [285]:
text = None
text_list = []
for i in range(len(df)):
    text = df['text'][i]
    text = re.sub('[^\w\s]', '', text)
    text = re.sub('\d', '', text)
    text = re.sub('[ \t]+' , ' ', text)
    text_list.append(text)
#     text_list.append(re.split(' ', text))

In [286]:
print(text_list[:5])

['Hope you are having a good week Just checking in', 'Kgive back my thanks', 'Am also doing in cbe only But have to pay', 'complimentary STAR Ibiza Holiday or Â cash needs your URGENT collection NOW from Landline not to lose out BoxSKWPPPM', 'okmail Dear Dave this is your final notice to collect your Tenerife Holiday or CASH award Call from landline TCs SAE Box CWWX ppm']


In [287]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=0., max_df=1.0)
X = vect.fit_transform(text_list)
# print(pd.DataFrame(X.A, columns=vect.get_feature_names()).to_string()) #데이터 프레임 필요 없음.
# df_text = pd.DataFrame(X.toarray().transpose(), index = vect.get_feature_names()) #데이터 프레임 필요 없음

In [288]:
X

<5559x8589 sparse matrix of type '<class 'numpy.int64'>'
	with 70442 stored elements in Compressed Sparse Row format>

In [289]:
X.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [290]:
X.shape

(5559, 8589)

In [291]:
df.shape

(5559, 2)

In [292]:
import numpy as np
y = np.array(df.type.map({'ham' : 0, 'spam' : 1}))

In [293]:
y.shape

(5559,)

In [294]:
y

array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

# Train, Test set 나누기

In [295]:
X.shape[0]*0.8

4447.2

In [296]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [297]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(X, test_size=0.3, random_state=42)
x_train.shape, x_test.shape

((3891, 8589), (1668, 8589))

In [298]:
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train.shape, y_test.shape

((3891,), (1668,))

In [268]:
# x_train, x_test = np.array(np.split(X.toarray(), [round(X.shape[0]*0.8,)]))
# y_train, y_test = np.array(np.split(y, [round(y.shape[0]*0.8,)]))

In [269]:
# x_train.shape, x_test.shape

In [270]:
# y_train.shape, y_test.shape

# 1. 베르누이 분포 나이브 베이즈

In [308]:
from sklearn.naive_bayes import BernoulliNB
model_bern = BernoulliNB().fit(x_train, y_train)

In [272]:
pre = model_bern.predict(x_test)
pre

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [275]:
print((pre==y_test).mean())

0.9664268585131894


# scikit-learn의 교차검증 기능

### 단순 데이터 분리¶
- 단순 데이터를 나누는 것과 교차검증의 차이 확인.

In [301]:
dfX_train, dfX_test, dfy_train, dfy_test = train_test_split(X, y, test_size=0.3, random_state=42)
dfX_train.shape, dfy_train.shape, dfX_test.shape, dfy_test.shape

((3891, 8589), (3891,), (1668, 8589), (1668,))

- 교차 검증의 장점은 train_test_split와 비교해보면 train_test_split는 데이터를 무작위로 나눔. 데이터를 무작위로 나눌 때 
- 훈련 세트 ==>  분류하기 어려운 샘플
- 테스트세트 ==> 분류하기 쉬운 샘플
- 테스트 세트의 정확도는 비현실적으로 높게 나올 것입니다. 반대의 경우라면 테스트 세트의 정확도는 비현실적으로 낮게 나옴

- 그러나 교차 검증을 사용하면 테스트 세트에 각 샘플이 정확하게 한번씩 들어가고 각 샘플은 폴드 중 하나에 속하며 각 폴드는 한번씩 테스트 세트가 됨 그렇기 때문에 교차 검증의 점수를 높이기 위해서는 데이터 셋에 있는 모든 샘플에 대해 모델이 잘 일반화 되어야 함

- 데이터를 여러개로 나누면 모델이 훈련 데이터에 얼마나 민감한지 알 수가 있음 
- 교차 검증의 주요 단점은 연산 비용이 늘어남 모델을 k개 만들어야 하므로 데이터를 한 번 나눴을때보다 대략 k배 더 느림

# K-fold 교차검증

- scikit-learn 패키지의 model_selection 서브 패키지는 KFold 클래스를 비롯한 다양한 교차검증 생성기를 제공한다. 이 생성기의 split 메서드는 학습용과 검증용의 데이터 인덱스를 출력하는 파이썬 반복자(iterator)를 반환한다.

In [320]:
from sklearn.naive_bayes import BernoulliNB
model_K = BernoulliNB().fit(X, y)

In [325]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# cv = KFold(5, shuffle=True, random_state=42)
cross_val_score(model_K, X, y, scoring="accuracy", cv=5) ##분류이므로 scoring = accuracy, 회기는 r2

array([0.98203055, 0.96945193, 0.9729973 , 0.9729973 , 0.96579658])

In [326]:
cv = KFold(5, shuffle=True, random_state=42)
cross_val_score(model_K, X, y, scoring="accuracy", cv=cv) #교차검증에서 데이터 전체를 넣어야 됨.

array([0.96852518, 0.97032374, 0.97571942, 0.97571942, 0.97659766])

In [327]:
cross_val_score(model_K, X, y, scoring="accuracy", cv=5).mean(), cross_val_score(model_K, X, y, scoring="accuracy", cv=cv).mean()

(0.9726547317804558, 0.97337708591003)

# 가우시안 정규 분포 나이브 베이즈